In [1]:
import time
import json
import pickle
import pandas as pd
from classes import *
from datetime import datetime
import plotly.graph_objects as go

In [2]:
with open("lpf_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)

In [12]:
def plot_data_arr(sensor_data, mat, sensor, labels):
    title = f"Matrix {mat}, Sensor {sensor}"
    heaters = sensor_data[f"mat_{mat}"][sensor]
    fig = go.Figure()

    for i, data in heaters.items():
        fig.add_trace(go.Scatter(x=data["Time Since PowerOn"],
                                 y=data["Filtered"],
                                 mode="lines",
                                 showlegend=False,
                                 name=f"H.S. {i}"))
    
    for label in labels[f"mat_{mat}"]:
        fig.add_vrect(x0=label["start"],
                      x1=label["end"],
                      annotation_text=classes[label["label"]],
                      annotation_position="top left",
                      fillcolor=colors[label["label"]],
                      opacity=0.2,
                      line_width=0)
    fig.update_yaxes(title_text="Resistance (Ohms)",
                     type="log", tickformat=".0e")
    fig.update_xaxes(title_text="Time since power on (ms)", tickformat=".0e")
    fig.update_layout(height=400, width=700,
                      title_x=0.5,
                      font_family="Times New Roman",
                      title_font_family="Times New Roman",
                      title=dict(text=title, pad=dict(t=0, r=0, b=0, l=0)),
                      margin=dict(t=50, r=10, b=0, l=0))

    fig.update_yaxes(type="log")
    fig.show()

In [4]:
def get_df(matrix):
    with open(f"raw_data/matrix_{matrix}/{matrix}.bmerawdata") as f:
        j = json.load(f)

    column_names = [el["name"] for el in j["rawDataBody"]["dataColumns"]]
    df = pd.DataFrame(j["rawDataBody"]["dataBlock"], columns=column_names)
    return df

In [5]:
def get_tspo_val(date_str, df):
    ts = time.mktime(datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S").timetuple())
    ts = int(ts)
    values = df[df["Real time clock"] == ts]["Time Since PowerOn"].values
    return values[0]

In [6]:
label_datetimes = {
    "mat_0": [
        {"start": "2024-10-12 12:22:00",
         "end": "2024-10-12 12:50:00",
         "cls": AIR},
        {"start": "2024-10-12 12:57:00",
         "end": "2024-10-12 13:20:01",
         "cls": OLIVE_100},
        {"start": "2024-10-12 13:47:00",
         "end": "2024-10-12 14:10:00",
         "cls": OLIVE_95},
        {"start": "2024-10-12 14:37:00",
         "end": "2024-10-12 15:00:00",
         "cls": OLIVE_90},
        {"start": "2024-10-12 15:27:00",
         "end": "2024-10-12 15:50:00",
         "cls": OLIVE_80},
        {"start": "2024-10-12 16:17:00",
         "end": "2024-10-12 16:40:00",
         "cls": OLIVE_70},
        {"start": "2024-10-12 17:12:00",
         "end": "2024-10-12 17:35:00",
         "cls": OLIVE_50},
        {"start": "2024-10-12 18:02:00",
         "end": "2024-10-12 18:25:00",
         "cls": OLIVE_0}
    ],
    "mat_1": [
        {"start": "2024-10-12 12:22:00",
         "end": "2024-10-12 12:50:00",
         "cls": AIR},
        {"start": "2024-10-12 12:57:00",
         "end": "2024-10-12 13:20:01",
         "cls": OLIVE_95},
        {"start": "2024-10-12 13:47:00",
         "end": "2024-10-12 14:10:00",
         "cls": OLIVE_90},
        {"start": "2024-10-12 14:37:00",
         "end": "2024-10-12 15:00:00",
         "cls": OLIVE_80},
        {"start": "2024-10-12 15:27:00",
         "end": "2024-10-12 15:50:00",
         "cls": OLIVE_70},
        {"start": "2024-10-12 16:17:00",
         "end": "2024-10-12 16:40:00",
         "cls": OLIVE_50},
        {"start": "2024-10-12 17:12:00",
         "end": "2024-10-12 17:35:00",
         "cls": OLIVE_0},
        {"start": "2024-10-12 18:02:00",
         "end": "2024-10-12 18:25:00",
         "cls": OLIVE_100}
    ]
}

sensor_labels = {"mat_0": [], "mat_1": []}
for matrix in range(2):
    df = get_df(matrix)
    for label_time in label_datetimes[f"mat_{matrix}"]:
        start_ts = get_tspo_val(label_time["start"], df)
        end_ts = get_tspo_val(label_time["end"], df)
        cls = label_time["cls"]
        sensor_labels[f"mat_{matrix}"].append({
            "start": start_ts,
            "end": end_ts,
            "label": cls
        })

In [22]:
plot_data_arr(sensor_data, 0, 0, sensor_labels)

In [9]:
with open("sensor_labels.pkl", "wb") as f:
    pickle.dump(sensor_labels, f)